In [ ]:
import time
import rtsvg
rt = rtsvg.RACETrack()
from xwords import XWords, XWordsSolver
import copy
import os
_dir_    = '../../../data/crossword_puzzle_screenshots/'
_files_  = os.listdir(_dir_)
entries_file    = None
geometries_file = None
blockers_file   = None
answers_file    = None
for _file_ in _files_:
    if   'entries'    in _file_: entries_file    = _dir_ + _file_
    elif 'geometries' in _file_: geometries_file = _dir_ + _file_
    elif 'blockers'   in _file_: blockers_file   = _dir_ + _file_
    elif 'answers'    in _file_: answers_file    = _dir_ + _file_
xwords = XWords(rt, entries_file, geometries_file, blockers_file, answers_file)
# xwords

In [ ]:
import os
import importlib.util
import inspect
def getClassesFromFile(file_path):
    classes = []
    module_name = os.path.splitext(os.path.basename(file_path))[0]
    spec        = importlib.util.spec_from_file_location(module_name, file_path)
    module      = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    for name, obj in inspect.getmembers(module):
        if inspect.isclass(obj) and obj.__module__ == module.__name__: classes.append(obj)
    return classes
models     = ['gemma3:4b'] # ['gemma3:1b', 'gemma3:4b', 'phi4', 'qwq', 'gemma3:27b', 'llama3.2', 'granite3.1-dense', 'qwen2.5:14b', 'deepseek-r1:32b']
results_lu = {}
for _filename_ in os.listdir():
    if _filename_.endswith('.py') == False: continue
    for _class_ in getClassesFromFile(_filename_):
        if issubclass(_class_, XWordsSolver):
            results_lu[_class_.__name__] = {}
            print(_class_)
            for _model_ in models:
                xwords_copy = copy.deepcopy(xwords)
                _instance_ = _class_(xwords=xwords_copy, model=_model_)
                answer_lu, request_stats, num_of_llm_requests = _instance_.solve()
                char_level_acc = xwords_copy.characterLevelAccuracy()
                results_lu[_class_.__name__][_model_] = (answer_lu, request_stats, num_of_llm_requests, char_level_acc, xwords_copy)
                print(f'\n{_class_.__name__} {_model_} {char_level_acc}')

In [ ]:
_xwords_ = results_lu['SweepAcrossContextDownSolver']['gemma3:4b'][4]
_xwords_

In [ ]:
import polars as pl
import networkx as nx
import types
def sweepClipGraphs(self):
    g_p, g_n = nx.Graph(), nx.Graph()
    for yi in range(self.y_tiles):
        for xi in range(self.x_tiles):
            _cell_ = self.cells[yi][xi]
            if _cell_.isBlocker() == False and len(_cell_.__guesses__) >= 2:
                _as_list_ = list(_cell_.__guesses__.keys())
                a_i, a_j     = _as_list_[0], _as_list_[1]
                a_i_u, a_j_v = _cell_.__guesses__[a_i], _cell_.__guesses__[a_j]
                if a_i_u == a_j_v: g_p.add_edge(a_i, a_j)
                else:              g_n.add_edge(a_i, a_j)
    return g_p, g_n
_xwords_.sweepClipGraphs = types.MethodType(sweepClipGraphs, _xwords_)
_gp_, _gn_ = _xwords_.sweepClipGraphs()

_lu_ = {'a_i':[], 'a_j':[]}
for _node_ in _gp_.nodes:
    _nbors_ = _gp_.neighbors(_node_)
    for _nbor_ in _nbors_:
        _lu_['a_i'].append(str(_node_))
        _lu_['a_j'].append(str(_nbor_))
pos = nx.spring_layout(_gp_)
_pos_ = {}
for k, v in pos.items(): _pos_[k] = (float(v[0]), float(v[1]))
pos = _pos_
_df_      = pl.DataFrame(_lu_)
ln_params = {'relationships': [('a_i', 'a_j')], 'pos':pos, 'node_size':'small'}
_igl_     = rt.interactiveGraphLayout(_df_, ln_params=ln_params, w=900, h=600)
_igl_